In [1]:
import pandas as pd
import numpy as np
import os, shutil
from fastai.vision import *
from fastai.widgets import ClassConfusion

In [2]:

metadata_path='/home/jupyter/covid-chestxray-dataset/metadata.csv'
df=pd.read_csv(metadata_path)

#types we're interested in
covid_patients=df['finding']=='COVID-19'
CT=df['view']=='CT'
PA=df['view']=='PA'
covid_new = df
# %%
df[covid_patients & CT].shape
df[covid_patients & PA].shape
# %%
PA_covid=df[covid_patients & PA]
Others=df[~covid_patients & PA]
covid_files=[files for files in PA_covid['filename']]
other_files=[files for files in Others['filename']]
print(type(covid_files))

<class 'list'>


In [3]:
test ={'test_files': ["03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg",
"01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg",
"1-s2.0-S0929664620300449-gr2_lrg-a.jpg",
"2C26F453-AF3B-4517-BB9E-802CF2179543.jpeg",
"1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg",
"171CB377-62FF-4B76-906C-F3787A01CB2E.jpeg",
"1-s2.0-S1684118220300608-main.pdf-002.jpg",
"1F6343EE-AFEC-4B7D-97F5-62797EE18767.jpeg",
"21DDEBFD-7F16-4E3E-8F90-CB1B8EE82828.jpeg"]}
test_files = (["03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg",
"01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg",
"1-s2.0-S0929664620300449-gr2_lrg-a.jpg",
"2C26F453-AF3B-4517-BB9E-802CF2179543.jpeg",
"1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg",
"171CB377-62FF-4B76-906C-F3787A01CB2E.jpeg",
"1-s2.0-S1684118220300608-main.pdf-002.jpg",
"1F6343EE-AFEC-4B7D-97F5-62797EE18767.jpeg",
"21DDEBFD-7F16-4E3E-8F90-CB1B8EE82828.jpeg"])
test_df =  pd.DataFrame(test, columns=['test_files'])

In [4]:
test_df.head(10)

,test_files
0,03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg
1,01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg
2,1-s2.0-S0929664620300449-gr2_lrg-a.jpg
3,2C26F453-AF3B-4517-BB9E-802CF2179543.jpeg
4,1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg
5,171CB377-62FF-4B76-906C-F3787A01CB2E.jpeg
6,1-s2.0-S1684118220300608-main.pdf-002.jpg
7,1F6343EE-AFEC-4B7D-97F5-62797EE18767.jpeg
8,21DDEBFD-7F16-4E3E-8F90-CB1B8EE82828.jpeg


In [5]:
path = '/home/jupyter/covid-chestxray-dataset'
destpath = path +'/data/covid', path +'/data/other', path +'/data/test'
srcpath = '/home/jupyter/covid-chestxray-dataset/images'

for root, dirs, files in os.walk(srcpath):
  print("root", root)
  if not os.path.isdir(destpath[0]):
    os.makedirs(destpath[0])
  if not os.path.isdir(destpath[1]):
    os.makedirs(destpath[1])
  if not os.path.isdir(destpath[2]):
    os.makedirs(destpath[2])
  for file in files:
    if file in covid_files:
      #print("here?")
      shutil.copy((root+'/'+file),destpath[0])
    if file in other_files:
      shutil.copy((root+'/'+file),destpath[1])
    if file in test_files:
      shutil.copy((root+'/'+file),destpath[2])

root /home/jupyter/covid-chestxray-dataset/images


In [6]:

print("Number of images in Other: {}".format(len(other_files)),"Number of images in Covid: {}".format(len(covid_files)),"Number of images in Test Set: {}".format(len(test_files)) )

Number of images in Other: 47 Number of images in Covid: 99 Number of images in Test Set: 9


In [7]:
## new data download
#! git clone https://github.com/Kaggle/kaggle-api
#! pip install kaggle
#! kaggle datasets download tasnimfatima/CoronaHack -Chest X-Ray-Dataset
#! kaggle datasets download -d praveengovi/coronahack-chest-xraydataset
#! sudo apt install unzip
#! unzip coronahack-chest-xraydataset.zip

In [8]:
metadata_path='Coronahack-Chest-XRay-Dataset/Chest_xray_Corona_Metadata.csv'
df_new=pd.read_csv(metadata_path)

#types we're interested in
covid_patients_new=df_new['Label_2_Virus_category']=='COVID-19'

PA_covid_new=df_new[covid_patients_new ]
Others_new=df_new[~covid_patients_new]
covid_files=[files for files in PA_covid_new['X_ray_image_name']]
other_files=[files for files in Others_new['X_ray_image_name']]
print(len(PA_covid_new))
print(df_new.shape)


path = '/home/jupyter/covid-chestxray-dataset'
destpath = path +'/data/covid', path +'/data/other', path +'/data/test'
srcpath = 'Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
new_files = pd.read_csv('Coronahack-Chest-XRay-Dataset/Chest_xray_Corona_Metadata.csv')

count = 0
for root, dirs, files in os.walk(srcpath):
      for file in files:
            if file in covid_files:
                count = count + 1;
                shutil.copy((root+'/'+file),destpath[0])
            if file in other_files:
                shutil.copy((root+'/'+file),destpath[1])
print(count)

58
(5910, 6)
58


In [9]:

classes=['covid','other']
tfms = get_transforms()

In [10]:
data = (ImageList.from_folder('/home/jupyter/covid-chestxray-dataset/data')
        .split_by_rand_pct(valid_pct=0.25)
        .label_from_folder()
        .transform(tfms, size = 512)
        .databunch()).normalize(imagenet_stats)

In [11]:
df = data.to_df()
df.shape

(4043, 2)

In [12]:
import sklearn
from sklearn.model_selection import StratifiedKFold

In [13]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1)
precision=Precision()
recall=Recall()
AUC=AUROC()

In [14]:
for train_index, val_index in skf.split(df.index, df['y']):
    data_fold = (ImageList.from_df(df, '/home/jupyter/covid-chestxray-dataset/data')
                .split_by_idxs(train_index, val_index)
                .label_from_df()
                .transform(tfms,size =  512)
                .databunch()).normalize(imagenet_stats)
    learn = cnn_learner(data_fold, models.resnet50, metrics=(accuracy,precision,recall,AUC)).to_fp16()
    learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,precision,recall,auroc,time
0,1.063116,0.729147,0.724351,0.377604,0.555203,0.618378,05:28


/opt/anaconda3/lib/python3.7/site-packages/fastai/metrics.py:191: UserWarning: average=`binary` was selected for a non binary case. Value for average has now been set to `macro` instead.
  warn("average=`binary` was selected for a non binary case. Value for average has now been set to `macro` instead.")


epoch,train_loss,valid_loss,accuracy,precision,recall,auroc,time
0,1.059726,0.912556,0.597033,0.360385,0.511439,0.671153,05:07


epoch,train_loss,valid_loss,accuracy,precision,recall,auroc,time
0,1.051656,0.603704,0.797281,0.395062,0.529318,0.492459,05:03


epoch,train_loss,valid_loss,accuracy,precision,recall,auroc,time
0,1.067493,0.566147,0.779703,0.363636,0.573792,0.655460,05:05


epoch,train_loss,valid_loss,accuracy,precision,recall,auroc,time
0,1.064930,1.068546,0.504951,0.356061,0.498099,0.861450,05:04
